In [1]:
import matplotlib.pyplot as plt
import math
import datetime
import networkx as nx
import os

In [2]:
def TodaysDate():
        
    today = datetime.date.today()
    TodaysDate = today.strftime('%d%b%Y')
    
    return TodaysDate

In [3]:
def DNACodingSequence(DNASequence, QualityScoreSequence, StartSequence, StopSequence):
#utilises ONLY ONE StopSequence, returns ONLY ONE CodingSequence
    
    QualityScoreString = """!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"""
    ThresholdQualityScore = 29 # ThresholdQualityScore must be between 0 and 93
    ThresholdQualityString = QualityScoreString[ThresholdQualityScore:]
    
    MinLength = 24
    MaxLength = 240
            
    StartIndex = DNASequence.find(StartSequence) + len(StartSequence)
    StopIndex = DNASequence.rfind(StopSequence)
    CodingSequence =  DNASequence[StartIndex:StopIndex]
    if MinLength <= len(CodingSequence) and len(CodingSequence) <= MaxLength and len(CodingSequence)%3 == 0:
        for Character in QualityScoreSequence[StartIndex:StopIndex]:
            if Character not in ThresholdQualityString:
                return None
        return str(CodingSequence)

In [4]:
def Translation(CodingSequence):
#translates DNA sequence

    TranslationCode = {
                    'AAA':'K','AAC':'N','AAG':'K','AAU':'N',
                    'ACA':'T','ACC':'T','ACG':'T','ACU':'T',
                    'AGA':'R','AGC':'S','AGG':'R','AGU':'S',
                    'AUA':'I','AUC':'I','AUG':'M','AUU':'I',
                    
                    'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H',
                    'CCA':'P','CCC':'P','CCG':'P','CCU':'P',
                    'CGA':'R','CGC':'R','CGG':'R','CGU':'R',
                    'CUA':'L','CUC':'L','CUG':'L','CUU':'L',
                    
                    'GAA':'E','GAC':'D','GAG':'E','GAU':'D',
                    'GCA':'A','GCC':'A','GCG':'A','GCU':'A',
                    'GGA':'G','GGC':'G','GGG':'G','GGU':'G',
                    'GUA':'V','GUC':'V','GUG':'V','GUU':'V',
                    
                    'UAA':'#','UAC':'Y','UAG':'*','UAU':'Y',
                    'UCA':'S','UCC':'S','UCG':'S','UCU':'S',
                    'UGA':'&','UGC':'C','UGG':'W','UGU':'C',
                    'UUA':'L','UUC':'F','UUG':'L','UUU':'F'
                        }
    # UAA (ochre) — #
    # UAG (amber) — *
    # UGA (opal) — &
                    
    TranscriptionCode = {'A':'A','C':'C','G':'G','T':'U','U':'T'}
      
    RNASequence = ''
    for Nucleotide in CodingSequence:
        RNASequence += TranscriptionCode.get(Nucleotide,'X')
    #converts DNA to RNA
    #print RNASequence
        
    Peptide = ''
    while len(RNASequence) != 0:
        Peptide += TranslationCode.get(RNASequence[0:3],'Do not fuck with me!')
        RNASequence = RNASequence[3:]
    return Peptide

In [5]:
def SingleRoundDNASummary(fastqFileLocation):
#returns a list of lists with DNA-sequences and their frequencies, sorted by frequency in descending order
    
    RawDataFile = open(fastqFileLocation, 'r')
    Lines = RawDataFile.readlines()
    RawDataFile.close
    
    #StartSequence = 'ATG' # Met codon
    #StopSequence = 'TAG' # amber stop codon
    
    StartSequence = 'TAATACGACTCACTATAGGGTTAACTTTAAGAAGGAGATATACATATG'    # NNK - T7g10M.F48 
    StopSequence = 'TGCGGCAGCGGCAGCGGCAGCTAGGACGGGGGGCGGAAA' #NNK - CGS3an13.R39 
    #StartSequence = 'TAATACGACTCACTATAGGGTTGAACTTTAAGTAGGAGATATATCCATG'   #NNU - T7-CH-F49
    #StopSequence = 'TGTGGGTCTGGGTCTGGGTCTTAGGACGGGGGGCGGAAA'  #NNU - CGS3-CH-R39
    
    SingleRoundDNASummary = {}
    #creates empty SingleSelectionRoundSummary dictionary to store the results from a single round of selection
    #SingleSelectionRoundSummary = {CodingSequence_YZ:    Occurence_YZ}
        
    #populates SingleSelectionRoundSummary dictionary with the results from a single round of selection
    for i,Line in enumerate(Lines):
        if StartSequence in Line and StopSequence in Line:
            CodingSequence = DNACodingSequence(Line, Lines[i + 2], StartSequence, StopSequence)
            if CodingSequence != None:
                if CodingSequence not in SingleRoundDNASummary:
                    SingleRoundDNASummary[CodingSequence] = 1
                else:
                    SingleRoundDNASummary[CodingSequence] += 1

    return SingleRoundDNASummary

In [6]:
def DNASelectionSummary(fastqDataFolderLocation):
# returns a SelectionSummary dictionary with the following structure
# SelectionSummary = {SelectionRound_X:    {CodingDNA_XYZ:    Occurence_XYZ}}

    DNASelectionSummary = {}
    # creates empty SelectionSummary dictionary to store the results from all the rounds of selection
          
    for file in os.listdir(fastqDataFolderLocation):
        
        FileLocation = os.path.join(fastqDataFolderLocation, file)
          
        if file.endswith('.fastq'): # this conditional is necessary; without it some shit appears in the beginning of the file list
            RoundNumberFirstDigit = file[file.find('.')-2]
            RoundNumberSecondDigit = file[file.find('.')-1]
            if RoundNumberFirstDigit == '0':
                RoundNumber = int(RoundNumberSecondDigit)
                #print RoundNumber
            elif RoundNumberFirstDigit != '0':
                RoundNumber = int(file[file.find('.')-2 : file.find('.')])
                #print RoundNumber
        #(1.A) extracts the round number from the file name (file name should have two digit number before full stop — '00.') 
                
            SelectionRoundSummary = SingleRoundDNASummary(FileLocation)
            #(1.B) extracts single round results 
                    
            DNASelectionSummary[RoundNumber] = SelectionRoundSummary
            #(1.C) populate ConcatenatedResultsList
            #print ConcatenatedResultsList
            
    return DNASelectionSummary

In [7]:
def HammingDistance(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '%'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '%'
    
    HammingDistance = 0
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            HammingDistance = HammingDistance
        else:
            HammingDistance = HammingDistance + 1
            
    return HammingDistance

In [8]:
def HammingDistanceBasedFormating(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '-'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '-'
    
    HammingDistance = 0
    FormatedSequence2 = ''
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            FormatedSequence2 += Sequence2[i].lower()
            HammingDistance = HammingDistance
        else:
            FormatedSequence2 += Sequence2[i]
            HammingDistance = HammingDistance + 1            
    return FormatedSequence2

In [9]:
def TotalReads_BY_Round(fastqDataFolderLocation):
    DNASummary = DNASelectionSummary(fastqDataFolderLocation)
    
    TotalReads_BY_Round = {}
    for Round in DNASummary:
        TotalReads_BY_Round[Round] = sum(DNASummary[Round].values())
        
    return TotalReads_BY_Round

In [10]:
def BaseRoundSortedDNAsList(fastqDataFolderLocation, BaseRoundIndex):
    DNASummary = DNASelectionSummary(fastqDataFolderLocation)  
            
    DNAsOccurences_IN_BaseRound = DNASummary[BaseRoundIndex]
    BaseRoundSortedDNAsList = sorted(DNAsOccurences_IN_BaseRound, key = DNAsOccurences_IN_BaseRound.get, reverse = True)
    
    return BaseRoundSortedDNAsList

In [11]:
def DNAsAppearances_BY_Round(BaseRoundSortedDNAsList, fastqDataFolderLocation):
    
    DNAsOccurences_BY_Round = DNASelectionSummary(fastqDataFolderLocation)
    
    DNAsAppearances_BY_Round = {}
    
    for DNA in BaseRoundSortedDNAsList:
        DNAsAppearances_BY_Round[DNA] = []
        for Round in DNAsOccurences_BY_Round:
            if DNA in DNAsOccurences_BY_Round[Round]:
                DNAsAppearances_BY_Round[DNA] += [Round]
    return DNAsAppearances_BY_Round

In [ ]:
def DNASummaryReport(fastqDataFolderLocation, BaseRoundIndex, NumberOfTopDNAs, DNASummaryReportFileName):
    
    today = TodaysDate() 
    
    DNASummaryFileNameCSV = str(today) + 'DNASummary' + DNASummaryReportFileName + '.csv'
    DNASummaryReportFile = open(DNASummaryFileNameCSV, 'w')
    
    DNASummary = DNASelectionSummary(fastqDataFolderLocation)
    TotalDNASummary = TotalReads_BY_Round(fastqDataFolderLocation)
    SortedRoundsList = sorted(DNASummary.keys())
    
    BaseRoundSortedDNAs = BaseRoundSortedDNAsList(fastqDataFolderLocation, BaseRoundIndex)
    BaseRoundTopSortedDNAs = BaseRoundSortedDNAs[0 : (NumberOfTopDNAs)] 
    
    DNASummaryReportFile.write('DNA sequence' + ',')
    for Round in SortedRoundsList:
        DNASummaryReportFile.write('round # ' + str(Round) + ' occurence (#)' + ',')
    DNASummaryReportFile.write('\n')
    
    for DNA in BaseRoundTopSortedDNAs:
        DNASummaryReportFile.write(DNA + ',')
        for Round in SortedRoundsList:
            DNASummaryReportFile.write(str(DNASummary[Round].get(DNA, 0)) + ',')
        DNASummaryReportFile.write('\n')
        
    DNASummaryReportFile.write('total #' + ',')
    for Round in SortedRoundsList:
        DNASummaryReportFile.write(str(TotalDNASummary[Round]) + ',')
    DNASummaryReportFile.write('\n\n\n')
    
    DNASummaryReportFile.write('DNA sequence' + ',')
    for Round in SortedRoundsList:
        DNASummaryReportFile.write('round # ' + str(Round) + ' fraction (%)' + ',')
    DNASummaryReportFile.write('\n')
    
    for DNA in BaseRoundTopSortedDNAs:
        DNASummaryReportFile.write(DNA + ',')
        for Round in SortedRoundsList:
            DNAFraction = float((DNASummary[Round].get(DNA, 0)))/float(TotalDNASummary[Round])
            DNASummaryReportFile.write('{:.3%}'.format(DNAFraction) + ',')
        DNASummaryReportFile.write('\n')
            
    DNASummaryReportFile.close()
    
#-------------------------------------------------------------------------------
   
    plt.style.use('fivethirtyeight') # just to create 'ggplot' style
    
    Xs = []
    Ys = []
    for DNA in BaseRoundTopSortedDNAs:
        DNAsFractions_BY_Round = []
        for Round in SortedRoundsList:
            DNAsFractions_BY_Round += [float((DNASummary[Round].get(DNA, 0)))/float(TotalDNASummary[Round])]
        
        x = SortedRoundsList
        y = DNAsFractions_BY_Round
        Xs += x
        Ys += y
        
        plt.plot(x, y,
                    'o-',
                    lw = 2.0,
                    ms = 4.0,
                    mew = 0.1,
                    mec = '#191919')
    
    XMin = min(Xs) - 0.05*(max(Xs) - min(Xs))
    XMax = max(Xs) + 0.05*(max(Xs) - min(Xs))
    YMin = min(Ys) - 0.05*(max(Ys) - min(Ys))
    YMax = max(Ys) + 0.05*(max(Ys) - min(Ys))
    
    plt.axis([XMin, XMax, YMin, YMax])
    
    plt.xlabel('Selection Round #', fontsize=14)
    plt.ylabel('DNA Fraction', fontsize=14)
    
    legend = plt.legend(BaseRoundTopSortedDNAs, loc='upper center', bbox_to_anchor=(0.5, -0.15),
                        fancybox=True, shadow=False, ncol=2)
    
    DNASummaryFileNamePNG = str(today) + 'DNASummary' + DNASummaryReportFileName + '.png'
    
    plt.savefig(DNASummaryFileNamePNG, bbox_extra_artists=[legend],bbox_inches='tight', dpi = 300)
    plt.show()
    plt.close()

In [ ]:
DataFolderLocation = '/Users/NikitaLoik/Documents/data/SelectionBias/AvsO/AvsOWith/SequencingResults'
BaseSelectionRoundNumber = 2
TopNDNAsNumber = 24
SummaryFileName = 'DNASequencesAnalysis'

DNASummaryReport(DataFolderLocation, BaseSelectionRoundNumber, TopNDNAsNumber, SummaryFileName)